In [2]:
from manim import *
import numpy as np
from ManimCENN import NeuralNetworkMobject

Manim Community v0.14.0

In [3]:
%%manim -v WARNING -qh --save_sections NeuralNetworkTest

class NeuralNetworkTest(Scene):
    def construct(self):
        # Display Bird Image / Title and then display details
        title = Text("Deep Reinforcement Learning", font_size = 45, color = BLUE)
        title.to_corner(UP + LEFT)
        title_ul = Underline(title, color = BLUE)
        bird_img = ImageMobject("assets/bird_img.png")
        bird_img.scale_to_fit_height(config.frame_height)
        bird_img.to_edge(RIGHT, buff=0)
        self.play(Write(title), Create(title_ul), FadeIn(bird_img))
        self.wait()
        self.next_section()
        
        bird_img_label = ImageMobject("assets/bird_img_label.png")
        bird_img_label.scale_to_fit_height(config.frame_height)
        bird_img_label.to_edge(RIGHT, buff=0)
        self.play(FadeIn(bird_img_label))
        self.play(FadeOut(bird_img))
        self.wait()
        self.next_section()
        
        # Create NN
        NN = NeuralNetworkMobject([3, 1],
                                  neuron_radius = 0.15,
                                  neuron_to_neuron_buff = MED_SMALL_BUFF,
                                  layer_to_layer_buff = LARGE_BUFF,
                                  output_neuron_color = WHITE,
                                  input_neuron_color = WHITE,
                                  hidden_layer_neuron_color = WHITE,
                                  neuron_stroke_width = 5,
                                  neuron_fill_color = WHITE,
                                  edge_color = RED,
                                  edge_stroke_width = 4,
                                  edge_propogation_color = YELLOW,
                                  edge_propogation_time = 1,
                                  max_shown_neurons = 16,
                                  brace_for_large_layers = True,
                                  average_shown_activation_of_large_layer = True,
                                  include_output_labels = False,
                                  arrow = False,
                                  arrow_tip_size = 0.1,
                                  left_size = 1,
                                  neuron_fill_opacity = 0)
        
        NN.scale(3)
        self.play(Create(NN.shift(2 * LEFT, DOWN)))
        self.wait(0.5)
        
        input_text_Group = []
        input_label_list = ["Bird Y", "Top-Pipe Distance", "Bottom-Pipe Distance"]
        for index, text_str in enumerate(input_label_list):
            neuron = NN.layers[0].neurons[index]
            neuron_width = neuron.get_width()
            
            text = Text(text_str, font_size = 45)
            text.move_to(neuron)
            text.to_corner(UP).shift(DOWN)

            input_text_Group.append(text)
        
        weight_tex_Group = []
        weight_label_list = ["$w_1$", "$w_2$", "$w_3$"]
        for index, tex_str in enumerate(weight_label_list):
            neuron = NN.layers[0].neurons[index]
            neuron_width = neuron.get_width()
            
            tex = Tex(tex_str, font_size = 45, color = RED)
            tex.move_to(neuron).shift(LEFT)

            weight_tex_Group.append(tex)
            
        NN_Edges = NN.edge_groups[0].copy()
        NN_Edges_Flash = [ShowPassingFlash(i.set_color(YELLOW), run_time=1, time_width=2) for i in NN_Edges]
        
        output_neuron = NN.layers[-1].neurons[0]
        output_neuron_text = Text("Output", font_size = 45)
        output_neuron_text.move_to(output_neuron)
        output_neuron_text.shift(UP)
        self.play(Write(output_neuron_text))
        self.wait()
        self.next_section()
        
        for i in range(0, 3):
            new_neuron = Circle(radius = 0.15, stroke_width = 5, stroke_color = WHITE, fill_color = WHITE, fill_opacity = 0.3)
            new_neuron.scale(3)
            new_neuron.move_to(NN.layers[0].neurons[i])
            
            self.play(Write(input_text_Group[i]))
            self.play(Transform(input_text_Group[i], new_neuron))
            self.play(Write(weight_tex_Group[i]))
            self.play(Transform(weight_tex_Group[i], NN.edge_groups[0].copy()[i]))
            self.remove(weight_tex_Group[i], NN.edge_groups[0].copy()[i])
            self.play(NN_Edges_Flash[i], FadeOut(input_text_Group[i], new_neuron))
            self.wait()
            self.next_section()
            
        self.wait()

<string>:54: DeprecationWarning: This method is not guaranteed to stay around. Please prefer getting the attribute normally.
<string>:66: DeprecationWarning: This method is not guaranteed to stay around. Please prefer getting the attribute normally.


In [3]:
%%manim -v WARNING -qh TanhFunctionPlot

# Helper Functions
def get_horizontal_line_to_graph(axes, function, x, width, color):
    result = VGroup()
    line = DashedLine(
        start=axes.c2p(0, function(x)),
        end=axes.c2p(x, function(x)),
        stroke_width=width,
        stroke_color=color,
    )
    dot = Dot().set_color(color).move_to(axes.c2p(x, function(x)))
    result.add(line, dot)
    return result

def get_vertical_line_to_graph(axes, function, x, width, color):
    result = VGroup()
    line = DashedLine(
        start=axes.c2p(x, 0),
        end=axes.c2p(x, function(x)),
        stroke_width=width,
        stroke_color=color,
    )
    dot = Dot().set_color(color).move_to(axes.c2p(x, function(x)))
    result.add(line, dot)
    return result

class TanhFunctionPlot(Scene):
    def construct(self):
        # Construct Axes eith labels
        axes = Axes(
            x_range=[-8.3, 8.3, 1],
            y_range=[-1.1, 1.1, 0.25],
            x_length=12,
            axis_config={"color": GREEN},
            x_axis_config={
                "numbers_to_include": np.arange(-8, 10, 2),
                "numbers_with_elongated_ticks": np.arange(-10, 10.01, 2),
            },
            y_axis_config={
                "numbers_to_include": np.arange(-1, 1.5, 0.5),
                "numbers_with_elongated_ticks": np.arange(-1, 1.5, 0.5),
            },
            tips=False,
        )
        axes_labels = axes.get_axis_labels()
        
        # Construct Function with labels
        tanh_f = lambda x: np.tanh(x)
        tanh_graph = axes.plot(lambda x: np.tanh(x), color=BLUE)
        tanh_label = axes.get_graph_label(
            tanh_graph, "\\tanh(x)", x_val=-10, direction=UP / 2
        )
        
        # Play Axes and Function
        plot = VGroup(axes, tanh_graph)
        labels = VGroup(axes_labels, tanh_label)
        self.play(Create(plot), Create(labels))
        self.wait()
        
        # Create ValueTracker to track our updaters
        k = ValueTracker(-5)  # Tracking the end values of stuff to show
        
        # Horizontal and Vertical lines (with dots)
        moving_h_line = always_redraw(
            lambda: get_horizontal_line_to_graph(
                axes=axes, function=tanh_f, x=k.get_value(), width=4, color=YELLOW
            )
        )
        moving_v_line = always_redraw(
            lambda: get_vertical_line_to_graph(
                axes=axes, function=tanh_f, x=k.get_value(), width=4, color=YELLOW
            )
        )
        
        # Inputs for lines
        input_xvalue_text = Text(f"Input: ", font_size = 40, color = YELLOW).to_corner(UP + LEFT)
        input_xvalue = always_redraw(
            lambda: DecimalNumber(num_decimal_places=2)
            .set_value(k.get_value())
            .next_to(input_xvalue_text, RIGHT, buff=0.1)
            .set_color(YELLOW)
        )
        
        # and Outputs for lines
        output_yvalue_text = Text(f"Output: ", font_size = 40, color = YELLOW).to_corner(UP + LEFT).shift(DOWN / 2)
        output_yvalue = always_redraw(
            lambda: DecimalNumber(num_decimal_places=2)
            .set_value(tanh_f(k.get_value()))
            .next_to(output_yvalue_text, RIGHT, buff=0.1)
            .set_color(YELLOW)
        )
        
        # add lines and inputs/outputs, and animate updaters
        updater_objects = VGroup(moving_h_line, moving_v_line, input_xvalue_text, input_xvalue, output_yvalue_text, output_yvalue)
        self.play(Create(updater_objects))
        self.play(k.animate.set_value(5), run_time=10, rate_func=linear)
        self.wait()